In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [15]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels)) # real loss
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a') # max
    ###################################################### ~Q(s,~a)= r # min
    ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
    targetQs = rewards + (gamma * aQs2_logits)
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
    eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # minimize Q
    aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # maximize Q
    eloss += tf.reduce_mean(targetQs) # minimize Q
    aloss2 += -tf.reduce_mean(targetQs) # maxizmie Q
    eloss += tf.reduce_mean(eQs_logits) # minimize Q
    aloss2 += -tf.reduce_mean(eQs_logits) # maxizmie Q
    return actions_logits, aloss, eloss, aloss2

In [16]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [17]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [18]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [19]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [20]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [21]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [22]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:14.0000 R:14.0000 rate:0.0280 aloss:0.7559 eloss:5.1474 aloss2:2.4151 exploreP:0.9986
Episode:1 meanR:24.5000 R:35.0000 rate:0.0700 aloss:0.7928 eloss:5.1544 aloss2:2.4208 exploreP:0.9952
Episode:2 meanR:19.6667 R:10.0000 rate:0.0200 aloss:0.7430 eloss:5.1330 aloss2:2.4238 exploreP:0.9942
Episode:3 meanR:23.7500 R:36.0000 rate:0.0720 aloss:0.7309 eloss:5.1336 aloss2:2.4282 exploreP:0.9906
Episode:4 meanR:21.2000 R:11.0000 rate:0.0220 aloss:0.7251 eloss:5.2239 aloss2:2.3243 exploreP:0.9896
Episode:5 meanR:21.8333 R:25.0000 rate:0.0500 aloss:0.7243 eloss:5.1636 aloss2:2.3889 exploreP:0.9871
Episode:6 meanR:20.4286 R:12.0000 rate:0.0240 aloss:0.7403 eloss:5.1463 aloss2:2.4096 exploreP:0.9859
Episode:7 meanR:20.0000 R:17.0000 rate:0.0340 aloss:0.7229 eloss:5.1901 aloss2:2.3646 exploreP:0.9843
Episode:8 meanR:20.6667 R:26.0000 rate:0.0520 aloss:0.7170 eloss:5.1769 aloss2:2.3872 exploreP:0.9818
Episode:9 meanR:20.6000 R:20.0000 rate:0.0400 aloss:0.7059 eloss:5.1979 aloss2:2.4

Episode:80 meanR:25.0247 R:33.0000 rate:0.0660 aloss:0.6906 eloss:4.8148 aloss2:3.3358 exploreP:0.8184
Episode:81 meanR:25.2927 R:47.0000 rate:0.0940 aloss:0.6892 eloss:4.8326 aloss2:3.4050 exploreP:0.8146
Episode:82 meanR:25.4096 R:35.0000 rate:0.0700 aloss:0.6915 eloss:4.8262 aloss2:3.4011 exploreP:0.8118
Episode:83 meanR:25.8214 R:60.0000 rate:0.1200 aloss:0.6855 eloss:4.8357 aloss2:3.3757 exploreP:0.8070
Episode:84 meanR:25.7647 R:21.0000 rate:0.0420 aloss:0.6898 eloss:4.8360 aloss2:3.3807 exploreP:0.8053
Episode:85 meanR:25.8605 R:34.0000 rate:0.0680 aloss:0.6915 eloss:4.8449 aloss2:3.5191 exploreP:0.8026
Episode:86 meanR:25.7356 R:15.0000 rate:0.0300 aloss:0.6803 eloss:4.8200 aloss2:3.3932 exploreP:0.8014
Episode:87 meanR:25.7386 R:26.0000 rate:0.0520 aloss:0.6844 eloss:4.8207 aloss2:3.4876 exploreP:0.7993
Episode:88 meanR:25.6854 R:21.0000 rate:0.0420 aloss:0.6825 eloss:4.8371 aloss2:3.4870 exploreP:0.7977
Episode:89 meanR:25.6222 R:20.0000 rate:0.0400 aloss:0.6909 eloss:4.8230 

Episode:159 meanR:38.2300 R:9.0000 rate:0.0180 aloss:0.6614 eloss:4.7755 aloss2:3.5648 exploreP:0.5926
Episode:160 meanR:38.8800 R:80.0000 rate:0.1600 aloss:0.6781 eloss:4.7880 aloss2:3.5738 exploreP:0.5880
Episode:161 meanR:39.8500 R:109.0000 rate:0.2180 aloss:0.6705 eloss:4.7786 aloss2:3.5504 exploreP:0.5817
Episode:162 meanR:40.7500 R:108.0000 rate:0.2160 aloss:0.6755 eloss:4.7699 aloss2:3.5649 exploreP:0.5756
Episode:163 meanR:42.1600 R:161.0000 rate:0.3220 aloss:0.6736 eloss:4.7739 aloss2:3.5796 exploreP:0.5665
Episode:164 meanR:41.7900 R:41.0000 rate:0.0820 aloss:0.6712 eloss:4.7727 aloss2:3.5813 exploreP:0.5642
Episode:165 meanR:43.4800 R:188.0000 rate:0.3760 aloss:0.6753 eloss:4.7768 aloss2:3.5895 exploreP:0.5539
Episode:166 meanR:44.3500 R:121.0000 rate:0.2420 aloss:0.6682 eloss:4.7741 aloss2:3.5733 exploreP:0.5474
Episode:167 meanR:44.8300 R:66.0000 rate:0.1320 aloss:0.6774 eloss:4.7880 aloss2:3.5963 exploreP:0.5438
Episode:168 meanR:45.6400 R:97.0000 rate:0.1940 aloss:0.6788

Episode:237 meanR:218.9500 R:282.0000 rate:0.5640 aloss:0.6651 eloss:4.7940 aloss2:3.5163 exploreP:0.0827
Episode:238 meanR:222.8500 R:500.0000 rate:1.0000 aloss:0.6667 eloss:4.7935 aloss2:3.5112 exploreP:0.0791
Episode:239 meanR:226.0500 R:379.0000 rate:0.7580 aloss:0.6755 eloss:4.7940 aloss2:3.5166 exploreP:0.0766
Episode:240 meanR:228.9900 R:311.0000 rate:0.6220 aloss:0.6638 eloss:4.7973 aloss2:3.5104 exploreP:0.0745
Episode:241 meanR:231.4800 R:282.0000 rate:0.5640 aloss:0.6719 eloss:4.7798 aloss2:3.5190 exploreP:0.0727
Episode:242 meanR:235.6800 R:500.0000 rate:1.0000 aloss:0.6611 eloss:4.8000 aloss2:3.5158 exploreP:0.0697
Episode:243 meanR:240.1400 R:500.0000 rate:1.0000 aloss:0.6674 eloss:4.7887 aloss2:3.5143 exploreP:0.0668
Episode:244 meanR:244.5300 R:500.0000 rate:1.0000 aloss:0.6678 eloss:4.7894 aloss2:3.5417 exploreP:0.0640
Episode:245 meanR:247.7800 R:348.0000 rate:0.6960 aloss:0.6533 eloss:4.8061 aloss2:3.5159 exploreP:0.0621
Episode:246 meanR:251.4800 R:390.0000 rate:0.7

Episode:315 meanR:440.5400 R:500.0000 rate:1.0000 aloss:0.5663 eloss:4.8526 aloss2:3.2906 exploreP:0.0121
Episode:316 meanR:441.2000 R:500.0000 rate:1.0000 aloss:0.5689 eloss:4.8597 aloss2:3.2726 exploreP:0.0120
Episode:317 meanR:441.2000 R:500.0000 rate:1.0000 aloss:0.5718 eloss:4.8472 aloss2:3.2888 exploreP:0.0119
Episode:318 meanR:442.7700 R:500.0000 rate:1.0000 aloss:0.5733 eloss:4.8475 aloss2:3.2787 exploreP:0.0118
Episode:319 meanR:444.3800 R:500.0000 rate:1.0000 aloss:0.5714 eloss:4.8438 aloss2:3.2944 exploreP:0.0117
Episode:320 meanR:446.0800 R:500.0000 rate:1.0000 aloss:0.5681 eloss:4.8486 aloss2:3.2946 exploreP:0.0116
Episode:321 meanR:447.9800 R:500.0000 rate:1.0000 aloss:0.5726 eloss:4.8424 aloss2:3.3069 exploreP:0.0116
Episode:322 meanR:450.1700 R:500.0000 rate:1.0000 aloss:0.5631 eloss:4.8481 aloss2:3.2899 exploreP:0.0115
Episode:323 meanR:450.1700 R:500.0000 rate:1.0000 aloss:0.5665 eloss:4.8428 aloss2:3.2972 exploreP:0.0114
Episode:324 meanR:452.5000 R:500.0000 rate:1.0

...

Episode:1312 meanR:244.4600 R:280.0000 rate:0.5600 aloss:0.4339 eloss:-36.1441 aloss2:52.5231 exploreP:0.0100
Episode:1313 meanR:244.5200 R:17.0000 rate:0.0340 aloss:0.4481 eloss:-39.4884 aloss2:56.0862 exploreP:0.0100
Episode:1314 meanR:243.2700 R:260.0000 rate:0.5200 aloss:0.4277 eloss:-36.5871 aloss2:53.1287 exploreP:0.0100
Episode:1315 meanR:243.2500 R:17.0000 rate:0.0340 aloss:0.4466 eloss:-34.3837 aloss2:50.4542 exploreP:0.0100
Episode:1316 meanR:242.6900 R:293.0000 rate:0.5860 aloss:0.4333 eloss:-36.6963 aloss2:53.1600 exploreP:0.0100
Episode:1317 meanR:242.1500 R:282.0000 rate:0.5640 aloss:0.4289 eloss:-36.6174 aloss2:52.9860 exploreP:0.0100
Episode:1318 meanR:240.9000 R:251.0000 rate:0.5020 aloss:0.4377 eloss:-38.1336 aloss2:54.5055 exploreP:0.0100
Episode:1319 meanR:243.5300 R:284.0000 rate:0.5680 aloss:0.4347 eloss:-40.2842 aloss2:56.7548 exploreP:0.0100
Episode:1320 meanR:243.2600 R:266.0000 rate:0.5320 aloss:0.4398 eloss:-38.7029 aloss2:55.3675 exploreP:0.0100
Episode:1321

Episode:1387 meanR:229.6000 R:184.0000 rate:0.3680 aloss:0.4443 eloss:-56.7375 aloss2:74.3907 exploreP:0.0100
Episode:1388 meanR:228.2000 R:81.0000 rate:0.1620 aloss:0.4245 eloss:-55.5369 aloss2:73.3837 exploreP:0.0100
Episode:1389 meanR:227.8700 R:265.0000 rate:0.5300 aloss:0.4452 eloss:-59.7769 aloss2:77.4110 exploreP:0.0100
Episode:1390 meanR:227.7100 R:263.0000 rate:0.5260 aloss:0.4277 eloss:-58.4443 aloss2:76.1542 exploreP:0.0100
Episode:1391 meanR:226.8900 R:243.0000 rate:0.4860 aloss:0.4291 eloss:-55.5666 aloss2:73.2239 exploreP:0.0100
Episode:1392 meanR:226.1400 R:244.0000 rate:0.4880 aloss:0.4281 eloss:-56.9829 aloss2:74.6249 exploreP:0.0100
Episode:1393 meanR:225.2500 R:204.0000 rate:0.4080 aloss:0.4262 eloss:-60.4247 aloss2:78.1680 exploreP:0.0100
Episode:1394 meanR:227.6500 R:252.0000 rate:0.5040 aloss:0.4295 eloss:-60.5818 aloss2:78.1941 exploreP:0.0100
Episode:1395 meanR:226.8400 R:204.0000 rate:0.4080 aloss:0.4171 eloss:-57.0656 aloss2:74.9651 exploreP:0.0100
Episode:139

Episode:1462 meanR:238.7400 R:243.0000 rate:0.4860 aloss:0.4232 eloss:-88.9957 aloss2:107.9357 exploreP:0.0100
Episode:1463 meanR:238.8200 R:256.0000 rate:0.5120 aloss:0.4430 eloss:-82.9815 aloss2:101.8525 exploreP:0.0100
Episode:1464 meanR:238.9900 R:257.0000 rate:0.5140 aloss:0.4048 eloss:-90.7798 aloss2:109.6839 exploreP:0.0100
Episode:1465 meanR:238.9800 R:228.0000 rate:0.4560 aloss:0.4357 eloss:-82.9114 aloss2:101.9256 exploreP:0.0100
Episode:1466 meanR:239.1600 R:240.0000 rate:0.4800 aloss:0.4272 eloss:-91.5487 aloss2:110.4909 exploreP:0.0100
Episode:1467 meanR:239.0400 R:224.0000 rate:0.4480 aloss:0.4311 eloss:-90.4908 aloss2:109.5061 exploreP:0.0100
Episode:1468 meanR:238.9300 R:242.0000 rate:0.4840 aloss:0.4302 eloss:-86.4378 aloss2:105.2947 exploreP:0.0100
Episode:1469 meanR:239.2500 R:279.0000 rate:0.5580 aloss:0.4145 eloss:-96.6010 aloss2:115.7671 exploreP:0.0100
Episode:1470 meanR:239.1900 R:250.0000 rate:0.5000 aloss:0.4300 eloss:-89.3416 aloss2:108.5603 exploreP:0.0100
E

Episode:1536 meanR:240.4500 R:260.0000 rate:0.5200 aloss:0.4479 eloss:-117.4971 aloss2:138.5980 exploreP:0.0100
Episode:1537 meanR:240.5800 R:231.0000 rate:0.4620 aloss:0.4598 eloss:-116.8768 aloss2:137.9825 exploreP:0.0100
Episode:1538 meanR:240.6000 R:254.0000 rate:0.5080 aloss:0.4501 eloss:-114.8502 aloss2:135.9490 exploreP:0.0100
Episode:1539 meanR:240.5300 R:237.0000 rate:0.4740 aloss:0.4600 eloss:-114.3032 aloss2:135.3406 exploreP:0.0100
Episode:1540 meanR:240.3300 R:233.0000 rate:0.4660 aloss:0.4600 eloss:-113.1856 aloss2:134.4807 exploreP:0.0100
Episode:1541 meanR:240.3500 R:234.0000 rate:0.4680 aloss:0.4731 eloss:-113.8293 aloss2:134.9518 exploreP:0.0100
Episode:1542 meanR:239.9100 R:235.0000 rate:0.4700 aloss:0.4708 eloss:-115.9047 aloss2:137.0471 exploreP:0.0100
Episode:1543 meanR:239.9000 R:236.0000 rate:0.4720 aloss:0.4515 eloss:-111.2093 aloss2:132.2382 exploreP:0.0100
Episode:1544 meanR:239.8000 R:242.0000 rate:0.4840 aloss:0.4650 eloss:-112.0122 aloss2:133.3229 exploreP

Episode:1610 meanR:195.5100 R:229.0000 rate:0.4580 aloss:0.4803 eloss:-126.8770 aloss2:150.0954 exploreP:0.0100
Episode:1611 meanR:195.4100 R:233.0000 rate:0.4660 aloss:0.4765 eloss:-125.5966 aloss2:148.7209 exploreP:0.0100
Episode:1612 meanR:193.9500 R:63.0000 rate:0.1260 aloss:0.4847 eloss:-132.6214 aloss2:155.6891 exploreP:0.0100
Episode:1613 meanR:193.9500 R:238.0000 rate:0.4760 aloss:0.4781 eloss:-133.4460 aloss2:156.6382 exploreP:0.0100
Episode:1614 meanR:192.2200 R:64.0000 rate:0.1280 aloss:0.5291 eloss:-127.9768 aloss2:150.8423 exploreP:0.0100
Episode:1615 meanR:192.1700 R:63.0000 rate:0.1260 aloss:0.5187 eloss:-131.2705 aloss2:154.0152 exploreP:0.0100
Episode:1616 meanR:192.1200 R:235.0000 rate:0.4700 aloss:0.4576 eloss:-142.3455 aloss2:165.5261 exploreP:0.0100
Episode:1617 meanR:190.8500 R:71.0000 rate:0.1420 aloss:0.5293 eloss:-121.9255 aloss2:144.8060 exploreP:0.0100
Episode:1618 meanR:189.1000 R:61.0000 rate:0.1220 aloss:0.4812 eloss:-127.8363 aloss2:151.0323 exploreP:0.01

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
